In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 921.9/921.9 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 784.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninsta

In [ ]:
import os
import shutil
import urllib.request
import xml.etree.ElementTree as ET
from PIL import Image
from pathlib import Path
from ultralytics import YOLO
import random

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
# # Load the license plates training dataset
# !gdown 1PBbW1I7Itdf83bH_p60vo3_jrGTS0yLM
# !unzip plate_image_with_dummy-train.zip -d plate_train

In [ ]:
# Unzip the dataset
!unzip '/content/drive/MyDrive/Colab Notebooks/Q2/plate_image_with_dummy-train.zip' -d plate_train

Streaming output truncated to the last 5000 lines.
  inflating: plate_train/train/dummy_te_14501.jpg  
  inflating: plate_train/train/dummy_te_14501.xml  
  inflating: plate_train/train/dummy_te_14502.jpg  
  inflating: plate_train/train/dummy_te_14502.xml  
  inflating: plate_train/train/dummy_te_14503.jpg  
  inflating: plate_train/train/dummy_te_14503.xml  
  inflating: plate_train/train/dummy_te_14504.jpg  
  inflating: plate_train/train/dummy_te_14504.xml  
  inflating: plate_train/train/dummy_te_14505.jpg  
  inflating: plate_train/train/dummy_te_14505.xml  
  inflating: plate_train/train/dummy_te_14506.jpg  
  inflating: plate_train/train/dummy_te_14506.xml  
  inflating: plate_train/train/dummy_te_14507.jpg  
  inflating: plate_train/train/dummy_te_14507.xml  
  inflating: plate_train/train/dummy_te_14508.jpg  
  inflating: plate_train/train/dummy_te_14508.xml  
  inflating: plate_train/train/dummy_te_14509.jpg  
  inflating: plate_train/train/dummy_te_14509.xml  
  inflating: 

In [ ]:
# Input paths
plate_train = '/content/plate_train/train'

In [ ]:
def organize_files(folder_path):
    # Define target directory
    images_dir = os.path.join(folder_path, 'images')
    labels_dir = os.path.join(folder_path, 'labels')

    # Create folders if don't exist
    os.makedirs(images_dir, exist_ok=True)
    os.makedirs(labels_dir, exist_ok=True)

    # Get list of files in the folder
    files = os.listdir(folder_path)

    # Image extensions to check
    image_extensions = ['.jpg', '.jpeg', '.png']
    # Label extensions to check
    label_extensions = ['.xml', '.txt']

    # Move files to appropriate folders
    for file in files:
        file_path = os.path.join(folder_path, file)

        # Skip if it's a directory
        if os.path.isdir(file_path):
            continue

        # Get file extension
        _, extension = os.path.splitext(file)
        extension = extension.lower()

        # Move images
        if extension in image_extensions:
            shutil.move(file_path, os.path.join(images_dir, file))

        # Move labels
        elif extension in label_extensions:
            shutil.move(file_path, os.path.join(labels_dir, file))

    print(f"✔ Successfully Organized {folder_path}")


# Apply organizer function
organize_files(plate_train)

✔ Successfully Organized /content/plate_train/train


In [ ]:
# Function to load classes from classes.txt file, which was stored from the LicensePlate_Detector.ipynb file
def load_classes_from_file(folder_path):
    classes = {}
    classes_file_path = os.path.join(folder_path, 'classes.txt')

    with open(classes_file_path, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                # Parse each line in format "class_id: class_name"
                parts = line.strip().split(':', 1)
                if len(parts) == 2:
                    class_id = int(parts[0].strip())
                    class_name = parts[1].strip()
                    classes[class_id] = class_name

    return classes


# Load classes dictionary from the file
classes = load_classes_from_file('/content/drive/MyDrive/Colab Notebooks/')

In [ ]:
# The process for converting labels to the acceptable of YOLO format
def convert_xml_to_txt(folder_path):
    global classes
    # Get paths to Images and Labels directories
    images_dir = os.path.join(folder_path, 'images')
    labels_dir = os.path.join(folder_path, 'labels')

    # Get list of XML files
    xml_files = [f for f in os.listdir(labels_dir) if f.endswith('.xml')]

    for xml_file in xml_files:
        # Parse XML file
        tree = ET.parse(os.path.join(labels_dir, xml_file))
        root = tree.getroot()

        # Get image dimensions
        size = root.find('size')
        if size is not None:
            width = float(size.find('width').text)
            height = float(size.find('height').text)
        else:
            # Get dimensions from image file
            img_file = xml_file.rsplit('.', 1)[0] + '.jpg'  # Assuming jpg extension
            img_path = os.path.join(images_dir, img_file)
            if os.path.exists(img_path):
                with Image.open(img_path) as img:
                    width, height = img.size
            else:
                print(f"Warning: Could not find image for {xml_file}")
                continue

        # Create output text file
        txt_file = xml_file.rsplit('.', 1)[0] + '.txt'
        txt_path = os.path.join(labels_dir, txt_file)

        with open(txt_path, 'w', encoding='utf-8') as f:
            # Process each object in the XML
            for obj in root.findall('object'):
                # Get class name
                class_name = obj.find('name').text

                # Handle special case where class_name is "ه‍"
                if class_name == "ه‍":
                    class_name = "ه"

                # Get class ID from dictionary
                class_id = [k for k, v in classes.items() if v == class_name][0]

                # Get bounding box coordinates
                bbox = obj.find('bndbox')
                xmin = float(bbox.find('xmin').text)
                ymin = float(bbox.find('ymin').text)
                xmax = float(bbox.find('xmax').text)
                ymax = float(bbox.find('ymax').text)

                # Convert to YOLO format (normalized)
                x_center = ((xmin + xmax) / 2) / width
                y_center = ((ymin + ymax) / 2) / height
                bbox_width = (xmax - xmin) / width
                bbox_height = (ymax - ymin) / height

                # Write to file with correct class ID
                f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {bbox_width:.6f} {bbox_height:.6f}\n")

        # Remove original XML file
        os.remove(os.path.join(labels_dir, xml_file))

    print(f"\n✔ Successfully Converted XML files in {folder_path}")
    print("\nCurrent Class Mappings:")

    for class_id, class_name in classes.items():
        print(f"Class {class_id}: {class_name}")


# Apply convert_xml_to_txt function
convert_xml_to_txt(plate_train)


✔ Successfully Converted XML files in /content/plate_train/train

Current Class Mappings:
Class 0: کل ناحیه پلاک
Class 1: 7
Class 2: 9
Class 3: ط
Class 4: 4
Class 5: 2
Class 6: 1
Class 7: 3
Class 8: 8
Class 9: س
Class 10: 5
Class 11: 6
Class 12: 0
Class 13: ل
Class 14: ت
Class 15: ج
Class 16: ص
Class 17: ب
Class 18: ه
Class 19: و
Class 20: ن
Class 21: م
Class 22: د
Class 23: ی
Class 24: ق
Class 25: ع
Class 26: ژ (معلولین و جانبازان)
Class 27: الف
Class 28: پ
Class 29: ث
Class 30: ظ
Class 31: S
Class 32: ک
Class 33: ف
Class 34: گ
Class 35: D
Class 36: تشریفات
Class 37: ش
Class 38: ز


In [ ]:
# Check the number of files
def count_files(folder):
    print(f"📂 {folder}:")
    print(" Images:", len(os.listdir(os.path.join(folder, "images"))))
    print(" Labels:", len(os.listdir(os.path.join(folder, "labels"))))

# Display the count of files by format
count_files(plate_train)

📂 /content/plate_train/train:
 Images: 45469
 Labels: 45469


In [ ]:
# Define validation data from the traninig dataset
def split_dataset(source_dir, train_ratio=0.8):
    images_dir = os.path.join(source_dir, 'images')
    labels_dir = os.path.join(source_dir, 'labels')

    image_files = [f for f in os.listdir(images_dir) if os.path.isfile(os.path.join(images_dir, f))]
    random.shuffle(image_files)

    train_split = int(len(image_files) * train_ratio)
    train_images = image_files[:train_split]
    val_images = image_files[train_split:]

    # Create train and val directories
    train_dir = os.path.join(source_dir, 'train')
    val_dir = os.path.join(source_dir, 'val')
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(val_dir, exist_ok=True)
    os.makedirs(os.path.join(train_dir, 'images'), exist_ok=True)
    os.makedirs(os.path.join(train_dir, 'labels'), exist_ok=True)
    os.makedirs(os.path.join(val_dir, 'images'), exist_ok=True)
    os.makedirs(os.path.join(val_dir, 'labels'), exist_ok=True)

    # Move images and corresponding labels
    for image_file in train_images:
        label_file = image_file.rsplit('.', 1)[0] + '.txt'
        shutil.move(os.path.join(images_dir, image_file), os.path.join(train_dir, 'images', image_file))
        shutil.move(os.path.join(labels_dir, label_file), os.path.join(train_dir, 'labels', label_file))
    for image_file in val_images:
        label_file = image_file.rsplit('.', 1)[0] + '.txt'
        shutil.move(os.path.join(images_dir, image_file), os.path.join(val_dir, 'images', image_file))
        shutil.move(os.path.join(labels_dir, label_file), os.path.join(val_dir, 'labels', label_file))

    print(f"✔ Dataset split complete.\n\tTrain images: {len(train_images)}\n\tValidation images: {len(val_images)}")


split_dataset(plate_train)

✔ Dataset split complete.
	Train images: 36375
	Validation images: 9094


In [ ]:
# Create a YAML file for dataset configuration
yaml_content = f"""train: /content/plate_train/train/train
val: /content/plate_train/train/val

nc: {len(classes)}
names: {classes}"""

# Save the YAML file
with open('char_dataset.yaml', "w") as f:
    f.write(yaml_content)

print(f"✔ Dataset configuration saved.")

✔ Dataset configuration saved.


In [ ]:
# Download the YOLOv11 weights
weights_dir = os.path.join(os.getcwd(), "weights")
os.makedirs(weights_dir, exist_ok=True)

url = "https://github.com/ultralytics/assets/releases/download/v8.3.0/yolo11n.pt"

file_name = os.path.join(weights_dir, os.path.basename(url))
urllib.request.urlretrieve(url, file_name)
print(f"✔ Downloaded successfully {file_name}")

✔ Downloaded successfully /content/weights/yolo11n.pt


In [ ]:
# Define YOLOv11 model process
models = {'nano':'/content/weights/yolo11n.pt'}

# Define the parameters for training
params = {
    'data': '/content/char_dataset.yaml',
    'project': '/content/drive/MyDrive/Colab Notebooks',    # Saving directory
    'name': 'character_detection',
    'save_period': 10,
    'classes': list(range(1, 40)),
    'epochs': 50,
    'imgsz': 256,
    'batch': 64,
    'patience': 10,
    'device': '0',
    'pretrained': True,
    'seed': 0,
    'optimizer': 'AdamW',
    'verbose': True,
    'save': True,
    'plots': True,
    'amp': True,
    'fraction':1,
    # Augmentation
    'hsv_h': 0.015,
    'hsv_s': 0.7,
    'hsv_v': 0.4,
    'translate': 0.1,
    'scale': 0.7,
    'fliplr': 0.5,
    'mosaic': 0.5,
    'mixup': 0.1,
    # Regularization
    'cos_lr': True,
    'lr0': 0.001,
    'lrf': 0.01,
    'weight_decay': 0.001  # L2
}

# Initialize model
model = YOLO('/content/weights/yolo11n.pt')

# Train the model
train_results = model.train(**params)

# Save the model
model.save("character_detection.pt")

# Evaluate the model
val_results = model.val(data='/content/char_dataset.yaml', name="character_detection")

# Display the results
print(f'mAP: {val_results.box.map}\n mAP50: {val_results.box.map50}\n mAP75: {val_results.box.map75}\n recall: {val_results.box.r}\n precision: {val_results.box.p}\n')

Ultralytics 8.3.80 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=/content/weights/yolo11n.pt, data=/content/char_dataset.yaml, epochs=50, time=None, patience=10, batch=64, imgsz=256, save=True, save_period=10, cache=False, device=0, workers=8, project=/content/drive/MyDrive/Colab Notebooks, name=character_detection6, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28,

train: Scanning /content/plate_train/train/train/labels.cache... 36373 images, 0 backgrounds, 14 corrupt: 100%|██████████| 36375/36375 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/plate_train/train/train/images/00538.jpg: ignoring corrupt image/label: image size (9, 47) <10 pixels
train: WARNING ⚠️ /content/plate_train/train/train/images/04015.jpg: ignoring corrupt image/label: image size (9, 33) <10 pixels
train: WARNING ⚠️ /content/plate_train/train/train/images/09693.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     25.701      27.354       1.125       26.08        27.5      1.0833      26.592      27.458        1.25]
train: WARNING ⚠️ /content/plate_train/train/train/images/09743.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [         28        33.5      2.7073      28.259      34.134      2.8049      28.476       35.61      2.8293      28.848      36.463      2.3415]
train: WARNING ⚠️ /content/plate_train/train/train/images/09781.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      11.91      5.6346      1.0385      12.106      5.7

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Scanning /content/plate_train/train/val/labels.cache... 9094 images, 0 backgrounds, 4 corrupt: 100%|██████████| 9094/9094 [00:00<?, ?it/s]

...

                   all       9090      72164      0.992      0.988      0.994      0.936
                     7       4755       6505      0.989      0.993      0.994      0.934
                     9       4769       6718      0.991      0.989      0.994      0.927
                     ط        454        454       0.99      0.996      0.995      0.908
                     4       4840       6621      0.989      0.987      0.994       0.93
                     2       4886       6852      0.982      0.967      0.992      0.928
                     1       5217       7353      0.991      0.987      0.994      0.893
                     3       4712       6564      0.976       0.98      0.993      0.935
                     8       4862       6800      0.992      0.992      0.994      0.929
                     س        424        424      0.951      0.964      0.988      0.867
                     5       4644       6386      0.996      0.996      0.995      0.936
                     

val: Scanning /content/plate_train/train/val/labels.cache... 9094 images, 0 backgrounds, 4 corrupt: 100%|██████████| 9094/9094 [00:00<?, ?it/s]

val: WARNING ⚠️ /content/plate_train/train/val/images/01537.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     2.2293      8.1229      2.4566      8.2571      2.7645      8.4857      3.0599        8.48      3.3058      8.5886      3.5393      8.6657      3.8058        8.86      4.0785      8.9286]
val: WARNING ⚠️ /content/plate_train/train/val/images/09624.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     12.189      5.3125       1.125      12.287       5.375      12.449      5.4271      1.1458      12.629      5.5729      1.0625      12.877      5.7917]
val: WARNING ⚠️ /content/plate_train/train/val/images/22751.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     5.2778      3.5862        5.45      3.5862      5.5556      3.6207      5.8056      3.7931]
val: WARNING ⚠️ /content/plate_train/train/val/images/27445.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordi


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 143/143 [00:48<00:00,  2.93it/s]


                   all       9090      72164      0.992      0.988      0.994      0.936
                     7       4755       6505      0.989      0.993      0.994      0.935
                     9       4769       6718      0.991      0.989      0.994      0.927
                     ط        454        454       0.99      0.996      0.995      0.908
                     4       4840       6621      0.989      0.987      0.994       0.93
                     2       4886       6852      0.982      0.967      0.993      0.927
                     1       5217       7353      0.991      0.987      0.994      0.894
                     3       4712       6564      0.976       0.98      0.993      0.935
                     8       4862       6800      0.992      0.992      0.994      0.929
                     س        424        424      0.951      0.964      0.988      0.866
                     5       4644       6386      0.996      0.996      0.995      0.936
                     